# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
%matplotlib inline

In [2]:
!pip install geoviews


In [3]:
!pip install hvplot geopandas panel


In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from geoviews import opts
import geoviews.tile_sources as gts
import geopandas as gpd
import holoviews as hv
import panel as pn

pn.extension()

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cooma,-36.2333,149.1333,5.20,75,5,0.61,AU,1730315884
1,1,atafu village,-8.5421,-172.5159,28.26,69,12,5.14,TK,1730315885
2,2,waitangi,-43.9535,-176.5597,11.57,83,23,1.79,NZ,1730315886
3,3,khanty-mansiysk,61.0042,69.0019,0.06,86,40,2.00,RU,1730315888
4,4,punta arenas,-53.1500,-70.9167,13.06,47,75,10.29,CL,1730315889
...,...,...,...,...,...,...,...,...,...,...
555,555,prakhon chai,14.6059,103.1208,21.97,94,99,2.63,TH,1730316540
556,556,xining,36.6167,101.7667,3.68,99,100,1.01,CN,1730316541
557,557,payyannur,12.1000,75.2000,24.26,90,10,2.11,IN,1730316542
558,558,berdigestyakh,62.0989,126.6992,-26.72,100,70,0.33,RU,1730316543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Define the plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [7]:
%%capture --no-display

# Define the plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.0,  # Adjust scale as needed
    color="City"
)

# Display the map plot
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
].dropna()

# Display sample data
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,bethel,41.3712,-73.4140,25.16,55,0,3.09,US,1730315844
42,42,adre,13.4667,22.2000,26.05,22,0,2.39,TD,1730315933
228,228,khuldabad,20.0500,75.1833,23.87,63,0,2.90,IN,1730316156
240,240,dirba,30.0667,75.9833,25.10,39,0,1.58,IN,1730316169
361,361,lompoc,34.6391,-120.4579,21.19,51,0,3.09,US,1730316314
384,384,tobruk,32.0836,23.9764,21.64,64,0,3.62,LY,1730316341
388,388,ghat,24.9647,10.1728,25.72,23,0,2.46,LY,1730316346
394,394,medina,24.4686,39.6142,26.24,44,0,1.03,SA,1730316239
452,452,san agustin,20.5333,-103.4667,26.62,21,0,0.00,MX,1730316422
454,454,port said,31.2565,32.2841,22.99,56,0,3.60,EG,1730316424


### Step 3: Create a new DataFrame called hotel_df

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,bethel,US,41.3712,-73.4140,55,
42,adre,TD,13.4667,22.2000,22,
228,khuldabad,IN,20.0500,75.1833,63,
240,dirba,IN,30.0667,75.9833,39,
361,lompoc,US,34.6391,-120.4579,51,
384,tobruk,LY,32.0836,23.9764,64,
388,ghat,LY,24.9647,10.1728,23,
394,medina,SA,24.4686,39.6142,44,
452,san agustin,MX,20.5333,-103.4667,21,
454,port said,EG,31.2565,32.2841,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.


In [10]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "apiKey": "ef848188d7a34a5cbd587988d32784f0", 
    "limit": 1, #first hotel only
    "categories": "accommodation.hotel"  # Filter for hotels
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionaryimport requests
    # Set parameters to search for a hotel

    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
adre - nearest hotel: No hotel found
khuldabad - nearest hotel: Hotel Kailas
dirba - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
ghat - nearest hotel: فندق تاسيلي
medina - nearest hotel: The Oberoi Madina
san agustin - nearest hotel: Microtel Inn & Suites by Wyndham Guadalajara Sur
port said - nearest hotel: The New Continental Hotel
plymouth - nearest hotel: Quality Inn
siwa oasis - nearest hotel: Flower of the city hotel
watertown - nearest hotel: Four Points by Sheraton
al ain city - nearest hotel: Al ain sanayya restaurant
tamanrasset - nearest hotel: فندق الأمان


,City,Country,Lat,Lng,Humidity,Hotel Name
26,bethel,US,41.3712,-73.4140,55,Hampton Inn Danbury
42,adre,TD,13.4667,22.2000,22,No hotel found
228,khuldabad,IN,20.0500,75.1833,63,Hotel Kailas
240,dirba,IN,30.0667,75.9833,39,No hotel found
361,lompoc,US,34.6391,-120.4579,51,Red Roof Inn Lompoc
384,tobruk,LY,32.0836,23.9764,64,فندق زهرة البطنان -طبرق
388,ghat,LY,24.9647,10.1728,23,فندق تاسيلي
394,medina,SA,24.4686,39.6142,44,The Oberoi Madina
452,san agustin,MX,20.5333,-103.4667,21,Microtel Inn & Suites by Wyndham Guadalajara Sur
454,port said,EG,31.2565,32.2841,56,The New Continental Hotel


In [11]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bethel,US,41.3712,-73.4140,55,Hampton Inn Danbury
1,adre,TD,13.4667,22.2000,22,No hotel found
2,khuldabad,IN,20.0500,75.1833,63,Hotel Kailas
3,dirba,IN,30.0667,75.9833,39,No hotel found
4,lompoc,US,34.6391,-120.4579,51,Red Roof Inn Lompoc
5,tobruk,LY,32.0836,23.9764,64,فندق زهرة البطنان -طبرق
6,ghat,LY,24.9647,10.1728,23,فندق تاسيلي
7,medina,SA,24.4686,39.6142,44,The Oberoi Madina
8,san agustin,MX,20.5333,-103.4667,21,Microtel Inn & Suites by Wyndham Guadalajara Sur
9,port said,EG,31.2565,32.2841,56,The New Continental Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map

In [13]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=2.0,  
    color="City", 
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)